## 数据准备

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
from skimage import measure

In [7]:
# change the corresponding paths to your local paths
land_mask = np.load("data/land_mask.npy")
targets = np.load("test_results/targets.npy")
times = np.load("test_results/times.npy")
sic_pred_ConvLSTM = np.load("test_results/sic_pred_ConvLSTM.npy")
sic_pred_PredRNN = np.load("test_results/sic_pred_PredRNN.npy")
sic_pred_SICFN = np.load("test_results/sic_pred_SICFN.npy")

In [8]:
pred_length = times.shape[0] // 2
pred_times = times[pred_length:]

pred_start_time = pred_times[0]
pred_end_time = pred_times[-1]

In [ ]:
pred_times

In [6]:
sic_pred_ConvLSTM = sic_pred_ConvLSTM[0, :, 0, :]
sic_pred_PredRNN = sic_pred_PredRNN[0, :, 0, :]
sic_pred_SICFN = sic_pred_SICFN[0, :, 0, :]

In [ ]:
sic_pred_ConvLSTM.shape, sic_pred_PredRNN.shape, sic_pred_SICFN.shape, targets.shape, land_mask.shape, times.shape

## 绘制结果的函数

In [6]:
# Function to format tick labels as percentages
def percentage(x, pos):
    return f"{x:.0%}"

In [7]:
# Function to plot sea ice concentration
def plot_sic(sic, cmap, save_path):
    os.makedirs(save_path, exist_ok=True)
    # 绘制海冰浓度图
    for i, time in enumerate(pred_times):
        fig, ax = plt.subplots()
        img = ax.imshow(sic[i], cmap=cmap, vmin=0, vmax=1)

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        # 显示colorbar
        cbar = plt.colorbar(img)
        cbar.ax.yaxis.set_major_formatter(FuncFormatter(percentage))

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

In [8]:
# Function to plot sea ice concentration difference
def plot_diff(sic, cmap, save_path):
    os.makedirs(save_path, exist_ok=True)

    # 绘制海冰浓度图
    for i, time in enumerate(pred_times):
        fig, ax = plt.subplots()
        img = ax.imshow(sic[i], cmap=cmap, vmin=-1, vmax=1)

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        # 显示colorbar
        cbar = plt.colorbar(img)
        cbar.ax.yaxis.set_major_formatter(FuncFormatter(percentage))

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

In [9]:
# 绘制SIE的函数
def plot_SIE(
    sic_pred_SICFN,
    sic_pred_PredRNN,
    sic_pred_ConvLSTM,
    targets,
    save_path,
):
    os.makedirs(save_path, exist_ok=True)

    # 绘制海冰SIE图
    for i, time in enumerate(pred_times):

        # 找到浓度大于0.15的部分
        thresholded_image_SICFN = sic_pred_SICFN[i] > 0.15
        thresholded_image_PredRNN = sic_pred_PredRNN[i] > 0.15
        thresholded_image_ConvLSTM = sic_pred_ConvLSTM[i] > 0.15
        thresholded_image_targets = targets[i] > 0.15

        # 计算边缘线
        contours_SICFN = measure.find_contours(thresholded_image_SICFN, 0.5)
        contours_PredRNN = measure.find_contours(thresholded_image_PredRNN, 0.5)
        contours_ConvLSTM = measure.find_contours(thresholded_image_ConvLSTM, 0.5)
        contours_targets = measure.find_contours(thresholded_image_targets, 0.5)

        # 创建新图
        fig, ax = plt.subplots()

        for contour in contours_SICFN:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#0080FF"
            )  # 指定线条颜色为红色

        for contour in contours_PredRNN:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#FF0000"
            )  # 指定线条颜色为绿色

        for contour in contours_ConvLSTM:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#00A000"
            )  # 指定线条颜色为蓝色

        for contour in contours_targets:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#000000"
            )  # 指定线条颜色为黑色

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

## 绘制SIC

In [ ]:
# 创建渐变色的colormap
colors = [(0, "#04629a"), (1, "white")]  # 定义颜色映射
cmap_ice_conc = LinearSegmentedColormap.from_list("cmap_ice_conc", colors)

In [ ]:
save_path = f"analysis_result/prediction/ConvLSTM/{pred_start_time}-{pred_end_time}"
plot_sic(sic_pred_ConvLSTM, cmap_ice_conc, save_path)

In [ ]:
save_path = f"analysis_result/prediction/PredRNN/{pred_start_time}-{pred_end_time}"
plot_sic(sic_pred_PredRNN, cmap_ice_conc, save_path)

In [ ]:
save_path = f"analysis_result/prediction/SICFN/{pred_start_time}-{pred_end_time}"
plot_sic(sic_pred_SICFN, cmap_ice_conc, save_path)

In [ ]:
save_path = f"analysis_result/ground_truth/{pred_start_time}-{pred_end_time}"
plot_sic(targets, cmap_ice_conc, save_path)

## 绘制SIC Difference

In [ ]:
# 创建自定义colormap
colors = [
    (0, "blue"),
    (0.5, "white"),
    (1, "red"),
]  # 透明色用 (1, 1, 1, 0) 表示
cmap_diff = LinearSegmentedColormap.from_list("cmap_diff", colors)

In [ ]:
diff_ConvLSTM = sic_pred_ConvLSTM - targets
save_path = f"analysis_result/difference/ConvLSTM/{pred_start_time}-{pred_end_time}"
plot_diff(diff_ConvLSTM, cmap_diff, save_path)

In [ ]:
diff_PredRNN = sic_pred_PredRNN - targets
save_path = f"analysis_result/difference/PredRNN/{pred_start_time}-{pred_end_time}"
plot_diff(diff_PredRNN, cmap_diff, save_path)

In [ ]:
diff_SICFN = sic_pred_SICFN - targets
save_path = f"analysis_result/difference/SICFN/{pred_start_time}-{pred_end_time}"
plot_diff(diff_SICFN, cmap_diff, save_path)

In [ ]:
diff_ConvLSTM_SICFN = np.abs(diff_ConvLSTM) - np.abs(diff_SICFN)
save_path = f"analysis_result/difference/sic_pred_ConvLSTM - sic_pred_SICFN/{pred_start_time}-{pred_end_time}"
plot_diff(diff_ConvLSTM_SICFN, cmap_diff, save_path)

In [ ]:
diff_PredRNN_SICFN = np.abs(diff_PredRNN) - np.abs(diff_SICFN)
save_path = f"analysis_result/difference/sic_pred_PredRNN - sic_pred_SICFN/{pred_start_time}-{pred_end_time}"
plot_diff(diff_PredRNN_SICFN, cmap_diff, save_path)

## 绘制SIE

In [10]:
save_path = f"analysis_result/SIE/{pred_start_time}-{pred_end_time}"
plot_SIE(sic_pred_SICFN, sic_pred_PredRNN, sic_pred_ConvLSTM, targets, save_path)